In [1]:
# 載入需要的套件
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
import random


In [10]:
import random
print(random.randrange(11))

0
